Hier werden alle Daten aus verschiedenen Quellen zu einem großen Dataframe zusammen gefasst. Das sind zum einen die Daten welche uns vom Kurs zu Verfügung gestellt wurden, als auch die daten welche wir selber noch gefunden haben.

In [38]:
#packete für data handling laden
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [39]:
#Einlesen der Basisidaten, welche wir zur verfügung gestellt bekommen haben
#Wetterdaten
wetter = pd.read_csv("../wetter.csv")
print(wetter.head())
print(wetter.tail())
print(wetter.shape)

#Umsatzdaten
umsatz = pd.read_csv("../umsatzdaten_gekuerzt.csv")
print(umsatz.head())
print(umsatz.tail())
print(umsatz.shape)

#Kieler Woche Daten
kiwo = pd.read_csv("../kiwo.csv")
print(kiwo.head())
print(kiwo.tail())
print(kiwo.shape)

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0  2012-01-01         8.0      9.8250                   14        58.0
1  2012-01-02         7.0      7.4375                   12         NaN
2  2012-01-03         8.0      5.5375                   18        63.0
3  2012-01-04         4.0      5.6875                   19        80.0
4  2012-01-05         6.0      5.3000                   23        80.0
           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
2596  2019-07-28         3.0     23.3500                   14         5.0
2597  2019-07-29         6.0     25.2500                    7        61.0
2598  2019-07-30         7.0     20.7375                    8        61.0
2599  2019-07-31         6.0     20.4500                    7        61.0
2600  2019-08-01         5.0     21.0625                    9        61.0
(2601, 5)
        Datum  Warengruppe      Umsatz
0  2013-07-01            1  148.828353
1  2013-07-02            1  159.793757
2  

Die Umsatzdaten sind im long format nach Warengruppen. Bedeutet, dass jedes Datum mehrmals vorkommt. Das ist zum mergen der Daten nicht ideal. Daher den dataframe vom long format ins wide format bringen. 
In diesem Zuge werden direkt auch die Warengruppen richtig benannt

In [40]:
#get the umsart data in the wide format
# Pivot the DataFrame
umsatz_shaped = umsatz.pivot(index='Datum', columns='Warengruppe', values='Umsatz')

# Reset the index to make 'date' a column again
umsatz_shaped.reset_index(inplace=True)

#get colums names right
umsatz_shaped.columns = ['Datum', 'Brot', 'Broetchen', 'Crossaint', 'Konditorei', 'Kuchen', 'Saisonbrot']

print(umsatz_shaped.head())
print(umsatz_shaped.tail())    
print(umsatz_shaped.shape)

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
           Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
1814  2018-07-27  198.225523  677.415844  333.774099   65.563734  297.867853   
1815  2018-07-28  214.470393  716.576825  371.819770   54.047422  290.024007   
1816  2018-07-29  129.801991  721.822447  433.820697  128.952852  335.823557   
1817  2018-07-30  148.541340  616.358562  348.770346   71.822977  324.965348   
1818  2018-07-31  123.075682  586.081666 

In [49]:
#aufräumen der Daten. NaN durch 0 ersetzen
umsatz_shaped.fillna(0, inplace=True)

#Eine Spalte hinzufügen in welcher der Gesamtumsatz steht
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Brot'] + umsatz_shaped['Broetchen'] + umsatz_shaped['Crossaint'] + umsatz_shaped['Konditorei'] + umsatz_shaped['Kuchen'] + umsatz_shaped['Saisonbrot']
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Umsatz_total'].astype(int)
print(umsatz_shaped.head())
print(umsatz_shaped.shape)

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  
0         0.0          1269  
1         0.0          1430  
2         0.0          1124  
3         0.0          1184  
4         0.0          1288  
(1819, 8)


In [ ]:
#nun können die Umsatzdaten mit den Wetterdaten zusammengeführt werden
#merge the data
data = pd.merge(umsatz_shaped,wetter, how='left', on='Datum')
print(data.head())
print(data.shape)

#es werden nun die Zeilen behalten, für welche es Umsatzdaten gibt

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0         0.0          1269         6.0     17.8375                 15.0   
1         0.0          1430         3.0     17.3125                 10.0   
2         0.0          1124         7.0     21.0750                  6.0   
3         0.0          1184         7.0     18.8500                  7.0   
4         0.0          1288         5.0     19.9750                 12.0   

   Wettercode  
0        20.0  
1         NaN  
2        61.0  
3        20.0  


In [51]:
# und nun noch die Kieler Woche Daten hinzufügen
data = pd.merge(data, kiwo, how='left', on='Datum')
print(data.head())
print(data.shape)

#im moment steht da eine 1.0 wenn es Kieler Woche ist und eine NaN wenn nicht. Das ändern wir jetzt
data['KielerWoche'] = data['KielerWoche'].fillna(0)
data['KielerWoche'] = data['KielerWoche'].astype(int)
print(data.head())	

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0         0.0          1269         6.0     17.8375                 15.0   
1         0.0          1430         3.0     17.3125                 10.0   
2         0.0          1124         7.0     21.0750                  6.0   
3         0.0          1184         7.0     18.8500                  7.0   
4         0.0          1288         5.0     19.9750                 12.0   

   Wettercode  KielerWoche  
0        20.0          NaN  
1         NaN         

Nun haben wir alle Basisidaten zusammen geführt. Jetzt können noch die Variablen, welche wir selber gesucht haben hinzugefügt werden.

In [52]:
#hinzufügen der Inflationsdaten
#einlesen der Daten
inflation = pd.read_csv("./Verbraucherpreisindex/vpi_daily.csv")
print(inflation.head())
print(inflation.tail())
print(inflation.shape)


        Datum   VPI
0  2013-07-01  98.9
1  2013-07-02  98.9
2  2013-07-03  98.9
3  2013-07-04  98.9
4  2013-07-05  98.9
           Datum    VPI
1852  2018-07-27  104.4
1853  2018-07-28  104.4
1854  2018-07-29  104.4
1855  2018-07-30  104.4
1856  2018-07-31  104.4
(1857, 2)


In [53]:
#mergen der Daten
data = pd.merge(data, inflation, how='left', on='Datum')    
print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0         0.0          1269         6.0     17.8375                 15.0   
1         0.0          1430         3.0     17.3125                 10.0   
2         0.0          1124         7.0     21.0750                  6.0   
3         0.0          1184         7.0     18.8500                  7.0   
4         0.0          1288         5.0     19.9750                 12.0   

   Wettercode  KielerWoche   VPI  
0        20.0            0  98.9  
1         

In [46]:
#hinzufügen der Kreuzfahrschfissdaten
schiffe = pd.read_csv("./01_Kreuzfahrtschiffe/kiel_all.csv")
print(schiffe.head())

#create a subset of the data
schiffe = schiffe[['Arrival_date', 'Departure_date','Ship']]
print(schiffe.head())

#format arrival and departure date
schiffe['Arrival_date'] = pd.to_datetime(schiffe['Arrival_date'])
#converte the date to a string for merging
schiffe['Arrival_date'] = schiffe['Arrival_date'].dt.strftime('%Y-%m-%d')
#schiffe['Departure_date'] = pd.to_datetime(schiffe['Departure_date'], format='%Y-%m-%d')
print(schiffe.head())

print(type(data['Datum'][0]))

  Weekday Arrival_date Departure_date      Ship       Pier
0      So     31.03.12       31.03.12  AIDAcara  Ostseekai
1      So     07.04.12       07.04.12  AIDAcara  Ostseekai
2      So     14.04.12       14.04.12  AIDAcara  Ostseekai
3      So     21.04.12       21.04.12  AIDAcara  Ostseekai
4      So     28.04.12       28.04.12  AIDAcara  Ostseekai
  Arrival_date Departure_date      Ship
0     31.03.12       31.03.12  AIDAcara
1     07.04.12       07.04.12  AIDAcara
2     14.04.12       14.04.12  AIDAcara
3     21.04.12       21.04.12  AIDAcara
4     28.04.12       28.04.12  AIDAcara
  Arrival_date Departure_date      Ship
0   2012-03-31       31.03.12  AIDAcara
1   2012-07-04       07.04.12  AIDAcara
2   2012-04-14       14.04.12  AIDAcara
3   2012-04-21       21.04.12  AIDAcara
4   2012-04-28       28.04.12  AIDAcara
<class 'str'>


/tmp/ipykernel_32987/1555327337.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schiffe['Arrival_date'] = pd.to_datetime(schiffe['Arrival_date'])


In [54]:
#merge the data
data = pd.merge(data, schiffe, how='left', left_on='Datum', right_on='Arrival_date', indicator=True)
print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Bewoelkung  Temperatur  Windgeschwindigkeit  \
0         0.0          1269         6.0     17.8375                 15.0   
1         0.0          1430         3.0     17.3125                 10.0   
2         0.0          1124         7.0     21.0750                  6.0   
3         0.0          1184         7.0     18.8500                  7.0   
4         0.0          1288         5.0     19.9750                 12.0   

   Wettercode  KielerWoche   VPI Arrival_date Departure_date        Ship  \
0   

In [ ]:
#Achtung an manchen Tagen kommen mehrere Schiffe an. Das müssen wir noch bereinigen	
data['Ship'] = data['Ship'].fillna(0)
data['Ship'] = data['Ship'].astype(int)
print(data.head())